In [1]:
!pip install firebase
!pip install jinja2
!rm -r '/content/funCloud' 2> /dev/null #remove folder on restart
!git clone https://github.com/AmitPerets/funCloud.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'funCloud'...
remote: Enumerating objects: 576, done.
remote: Counting objects: 100% (226/226), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 576 (delta 92), reused 161 (delta 65), pack-reused 350
Receiving objects: 100% (576/576), 145.22 KiB | 5.81 MiB/s, done.
Resolving deltas: 100% (228/228), done.


In [2]:
from firebase import firebase
import google.colab.output as output
from IPython.display import JSON

class DB_Connector():
  connection = firebase.FirebaseApplication('https://puzzleton-45589-default-rtdb.firebaseio.com/',None)
  user = None
  name = None
  cuurent_index = -1

  @classmethod
  def login(cls,username,password):
    try:
      cls.user = cls.connection.get(f"/users/{username}",None)

      if cls.user["password"] == password:
        cls.name = username
        return True
    except:
      return False

  @classmethod
  def getUser(cls):
    return JSON({"name":cls.name,"admin":cls.user["admin"],"score":cls.user["score"],"sessions":cls.user["sessions"]})

  @classmethod
  def getUserName(cls):
    return cls.name

  @classmethod
  def isAdmin(cls):
    return cls.user["admin"]

  @classmethod
  def getUserSessions(cls):
    return JSON(cls.user["sessions"])

  @classmethod
  def getUserScore(cls):
    return cls.user["score"]

  @classmethod
  def updateSession(cls,score):
    try:
        cls.connection.put(f"/users/{cls.name}","score",cls.user["score"]+score)
        cls.connection.put(f"/users/{cls.name}/sessions",str(len(cls.user["sessions"])),score)
        cls.updateUser()
    except:
        return False

  @classmethod
  def updateUser(cls):
    cls.user = cls.connection.get(f"/users/{cls.name}",None)

  @classmethod
  def logout(cls):
     cls.user = None

  @classmethod
  def addQuest(cls,data):
    try:
      cls.connection.put("/questions",str(len(cls.getAllQuestions().keys())),data)
      return True
    except:
      return False

  @classmethod
  def getAllTitles(cls):
    questions = cls.connection.get("/questions",None)
    return JSON([question["title"] for question in questions])

  @classmethod
  def update_question(cls,title,options,correct):
    data = {"title":title,"correct":correct}
    for i,option in enumerate(options):
      data.update({"Option"+str(i+1):option})
    cls.connection.put(f"/questions",str(cls.current_index),data)

  @classmethod
  def getQuestion(cls,title):
    questions = cls.connection.get("/questions",None)
    for i,question in enumerate(questions):
      if question["title"] == title:
        cls.current_index = i
        return JSON(question)

  @classmethod
  def getAllQuestions(cls):
    return  cls.connection.get("/questions",None)

output.register_callback('db.login',DB_Connector.login)
output.register_callback('db.logout',DB_Connector.logout)
output.register_callback('db.update_question',DB_Connector.update_question)
output.register_callback('db.isAdmin',DB_Connector.isAdmin)
output.register_callback('db.getUser',DB_Connector.getUser)
output.register_callback('db.getUserName',DB_Connector.getUserName)
output.register_callback('db.getUserSessions',DB_Connector.getUserSessions)
output.register_callback('db.getUserScore',DB_Connector.getUserScore)
output.register_callback('db.addQuest',DB_Connector.addQuest)
output.register_callback('db.getAllTitles',DB_Connector.getAllTitles)
output.register_callback('db.getQuestion',DB_Connector.getQuestion)
output.register_callback('db.getAllQuestions',DB_Connector.getAllQuestions)

In [3]:
import random
from IPython.display import JSON
questions = DB_Connector.getAllQuestions()

asked = set()
answers = []
score = 0

def generate_question():
  if len(asked) >= len(questions):
    return None
  question = random.choice(questions)
  while question["title"] in asked:
    question = random.choice(questions)
  asked.add(question["title"])
  return JSON(question)


def submit_question(title,answer,type_):
  global score
  answers.append({'title':title,'answer':answer,'type':type_})
  if (type_ == "correct-answer"):
    score+=5

def get_score():
  return score

def graph_values():
  wrong = 0
  correct = 0
  for answer in answers:
    if answer["type"] == "wrong-answer":
      wrong+=1
    else:
      correct+=1
  return JSON({"correct":correct,"wrong":wrong})

def view_timeline():
  DB_Connector.updateSession(score);
  show_page('timeline',answers)

def clear():
  global asked
  global answers
  global score
  asked = set()
  answers = []
  score = 0

def progress():
    percentage = (len(asked) / len(questions)) * 100
    return percentage

output.register_callback('progress',progress)
output.register_callback('get_score',get_score)
output.register_callback('graph_values',graph_values)
output.register_callback('clear',clear)
output.register_callback('submit_question',submit_question)
output.register_callback('view_timeline',view_timeline)
output.register_callback('generate_question',generate_question)

In [4]:
from IPython.display import HTML, Javascript, clear_output
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader('/content/funCloud/html'))

def show_page(file_name, options=None):
    clear_output()
    template = env.get_template(f'{file_name}.html') #load page
    html = HTML(template.render(options=options)) #render page
    display(html) #display after rendering page
    display(Javascript(f'/content/funCloud/js/utils.js')) #load utils.js
    display(Javascript(f'/content/funCloud/js/{file_name}.js')) #load page.js


output.register_callback("show_page", show_page)
show_page("login")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>